In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('data/dynamically-generated-hate-speech.csv')

# Extract the relevant columns
texts = data['text']
labels = data['label'].apply(lambda x: 1 if x == "hate" else 0)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the vectorizer on the texts
text_features = tfidf_vectorizer.fit_transform(texts)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text_features, labels, test_size=0.2, random_state=42)

# Define the alpha range
alpha_range = [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10]
alpha_scores = []

# Iterate over each alpha value and evaluate using cross-validation
for alpha in alpha_range:
    nb = MultinomialNB(alpha=alpha)
    scores = cross_val_score(nb, X_train, y_train, cv=12, scoring='accuracy')
    alpha_scores.append(scores.mean())

# Print the results
for alpha, score in zip(alpha_range, alpha_scores):
    print(f"Alpha: {alpha}, Accuracy: {score:.4f}")

best_index = alpha_scores.index(max(alpha_scores))
best_alpha = alpha_range[best_index]
best_score = alpha_scores[best_index]

print(f"\nBest alpha value: {best_alpha}\n")

# Initializing and training the Naive Bayes classifier
nb_classifier = MultinomialNB(alpha=best_alpha)
nb_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Alpha: 0.001, Accuracy: 0.6608
Alpha: 0.01, Accuracy: 0.6608
Alpha: 0.1, Accuracy: 0.6614
Alpha: 0.5, Accuracy: 0.6639
Alpha: 1, Accuracy: 0.6646
Alpha: 2, Accuracy: 0.6656
Alpha: 5, Accuracy: 0.6624
Alpha: 10, Accuracy: 0.6537

Best alpha value: 2

Accuracy: 0.6749538461538461
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.52      0.60      3724
           1       0.67      0.80      0.73      4401

    accuracy                           0.67      8125
   macro avg       0.68      0.66      0.66      8125
weighted avg       0.68      0.67      0.67      8125

